# 240311_KS274_ASTARR_Output_Nextseq - starr_seq
This notebook will create all the necessary files, scripts and folders to pre-process the aforementioned project. Is designed to be used in a jupyter server deployed in a system running SLURM. The majority of the scripts and heavy-lifting processes are wrapped up in sbatch scripts.As an end user, in order to pre-process your samples provided in the spread sheet, you will simply need to *run the entire notebook* (Cell > Run all) and the system should take care of the rest for you.
#### Create necessary folder(s)

In [34]:
%%bash
mkdir -p /home/ks476/data_keith/encode4_duke//data/starr_seq/metadata
mkdir -p /home/ks476/data_keith/encode4_duke//data/starr_seq/raw_reads
mkdir -p /home/ks476/data_keith/encode4_duke//data/starr_seq/processed_raw_reads
mkdir -p /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts
mkdir -p /home/ks476/data_keith/encode4_duke//processing/starr_seq/jsons
mkdir -p /home/ks476/data_keith/encode4_duke//processing/starr_seq/logs

Save metadata file

In [1]:
%%writefile /home/ks476/data_keith/encode4_duke//data/starr_seq/metadata/starr_seq_download_metadata.240311_KS274_ASTARR_Output_Nextseq.txt
Sequencing core user	Sequencing core project	Sequencing core library name	name	Paired-end or single-end	Genome	Library type	Blacklist removal	Umis	I5 index
	240311_KS274_ASTARR_Output_Nextseq	KS275-Th17-ASTARR-Th17inp-Output-rep1	KS275_Th17_ASTARR_Th17inp_Output_rep1	PE	mm10	STARR-seq	True	True	4
	240311_KS274_ASTARR_Output_Nextseq	K562-ASTARR-repeat-rep1	K562_ASTARR_repeat_rep1	PE	hg38	STARR-seq	True	True	1
	240311_KS274_ASTARR_Output_Nextseq	K562-ASTARR-repeat-rep2	K562_ASTARR_repeat_rep2	PE	hg38	STARR-seq	True	True	13
	240311_KS274_ASTARR_Output_Nextseq	K562-ASTARR-repeat-rep3	K562_ASTARR_repeat_rep3	PE	hg38	STARR-seq	True	True	18
	240311_KS274_ASTARR_Output_Nextseq	KS258-Th17-ASTARR-Pooled-Output-rep1	KS258_Th1_ASTARR_Pooled_Output_rep1	PE	mm10	STARR-seq	True	True	17


Overwriting /home/ks476/data_keith/encode4_duke//data/starr_seq/metadata/starr_seq_download_metadata.240311_KS274_ASTARR_Output_Nextseq.txt


#### Merge lanes of FASTQ files

In [2]:
%%writefile /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/merge_lanes_240311_KS274_ASTARR_Output_Nextseq.sh
#!/bin/bash
#SBATCH --array=0-5%20
ORDER=240311_KS274_ASTARR_Output_Nextseq
RAW_DATA_DIR=/home/ks476/data_keith/encode4_duke//data/starr_seq/raw_reads/${ORDER}
PROCESSED_DATA_DIR=/home/ks476/data_keith/encode4_duke//data/starr_seq/processed_raw_reads/${ORDER}
METADATA=/home/ks476/data_keith/encode4_duke//data/starr_seq/metadata/starr_seq_download_metadata.240311_KS274_ASTARR_Output_Nextseq.txt

mkdir -p ${PROCESSED_DATA_DIR}
cd ${PROCESSED_DATA_DIR}

seq_name_header=$(/bin/grep -Eoi "sequencing.?core.?library.?name" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Sequencing core library name not found in ${METADATA}"
    exit 1
fi

name_header=$(/bin/grep -Poi "\tname\t" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Library Name column not found in ${METADATA}"
    exit 1
fi
name_header=$(echo ${name_header} | cut -f2)

seq_type_header=$(head -1 ${METADATA} | /bin/grep -Poi "paired.?end.?or.?single.?end")
if [[ $? == 1 ]];
then
    echo -e "ERROR: Paired-end or single-end column not found in ${METADATA}"
    exit 1
fi

sample_seq_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
sample_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
seq_type=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_type_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');


for read_pair in R1 R2 UMI;
do
    sample_files=$(/bin/ls ${RAW_DATA_DIR}/${sample_seq_name/ /}_S[0-9]*_L[0-9][0-9][0-9]_${read_pair}_* 2> /dev/null)
    if [[ $? != 0 ]]; # If no samples found with that read_pair, continue
    then
        continue;
    fi
    if [[ ${read_pair} == "R1" || (${seq_type/ /} == "PE" || ${seq_type/ /} == "pe") ]];
    then
        # Merge all lanes
        merged=$(basename $(echo ${sample_files} | awk '{print $1}') | sed -e 's/_L[0-9]\{3\}_/_/')
        cat ${sample_files} > ${merged};

        # Rename samples with our sample Names
        dest_filename=$(basename $(echo ${merged} | awk '{print $1}') | sed -r 's/\_S[0-9]+//; s/\_(R1|R2|UMI)\_/\.\1\./; s/\.[0-9]+\.fastq/\.fastq/')
        mv ${merged} ${dest_filename}

        cleaned_dest_filename=${dest_filename/${sample_seq_name/ /}/${sample_name/ /}}

        if [[ ${seq_type/ /} == "SE" || ${seq_type/ /} == "se" ]];
        then
            cleaned_dest_filename=${cleaned_dest_filename/.R1/}
        fi
        
        mv ${dest_filename} ${cleaned_dest_filename}
    fi
done


Overwriting /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/merge_lanes_240311_KS274_ASTARR_Output_Nextseq.sh


Execute file to merge lanes of FASTQ files

In [3]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -o /home/ks476/data_keith/encode4_duke//processing/starr_seq/logs/240311_KS274_ASTARR_Output_Nextseq_merge_fastq_files_%a.out \
 -p all \
 --array 0-4%20 \
 /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/merge_lanes_240311_KS274_ASTARR_Output_Nextseq.sh

Extract blocking job id

In [4]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create JSON files for CWL pipeline files

In [5]:
%%writefile /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/cwl_json_gen_240311_KS274_ASTARR_Output_Nextseq.sh
#!/bin/bash
ORDER=240311_KS274_ASTARR_Output_Nextseq
PROCESSED_DATA_DIR=/home/ks476/data_keith/encode4_duke//data/starr_seq/processed_raw_reads/${ORDER}
METADATA=/home/ks476/data_keith/encode4_duke//data/starr_seq/metadata/starr_seq_download_metadata.240311_KS274_ASTARR_Output_Nextseq.txt

python /data/reddylab/software/cwl/GGR-cwl/v1.0/json-generator/run.py \
    -m ${METADATA} \
    -d ${PROCESSED_DATA_DIR} \
    -o /home/ks476/data_keith/encode4_duke//processing/starr_seq/jsons \
    -t starr-seq \
    --fastq-gzipped \
    --mem 128000 \
    --nthreads 24 \
    --separate-jsons



Overwriting /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/cwl_json_gen_240311_KS274_ASTARR_Output_Nextseq.sh


Execute file to create JSON files

In [6]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -o /home/ks476/data_keith/encode4_duke//processing/starr_seq/logs/240311_KS274_ASTARR_Output_Nextseq_cwl_json_gen.out \
 -p all \
 --depend afterok:$1 \
 /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/cwl_json_gen_240311_KS274_ASTARR_Output_Nextseq.sh

Extract blocking job id

In [7]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create SLURM array master bash file for pe-umis samples

In [9]:
%%writefile /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/240311_KS274_ASTARR_Output_Nextseq-pe-umis.sh
#!/bin/bash
#SBATCH --job-name=cwl_starr_seq
#SBATCH --output=/home/ks476/data_keith/encode4_duke//processing/starr_seq/logs/240311_KS274_ASTARR_Output_Nextseq-pe-umis-%a.out
#SBATCH --mail-user=ks476@duke.edu
#SBATCH --mail-type=FAIL,END
#SBATCH --mem=128000
#SBATCH --cpus-per-task=24

export PATH="/data/reddylab/software/bin:$PATH"
export PATH="/data/reddylab/software/cwl/bin:$PATH"
export PATH="/data/reddylab/software/preseq_v2.0:$PATH"
export PATH="/data/reddylab/software/rsem-1.2.21/:$PATH"
export PATH="/data/reddylab/software/phantompeakqualtools-1.2/:$PATH"
export PATH="/data/reddylab/software/miniconda2/envs/cwl10/bin:$PATH"

module load bedtools2
module load fastqc
module load samtools
module load bowtie2
module load java

# For Fastqc
export DISPLAY=:0.0

# Make sure temporary files and folders are created in a specific folder
mkdir -p /home/ks476/data_keith/encode4_duke//processing/starr_seq/tmpdirs/tmp-240311_KS274_ASTARR_Output_Nextseq-pe-umis-${SLURM_ARRAY_TASK_ID}-
export TMPDIR="/home/ks476/data_keith/encode4_duke//processing/starr_seq/tmpdirs/tmp-240311_KS274_ASTARR_Output_Nextseq-pe-umis-${SLURM_ARRAY_TASK_ID}-"

cwltool --debug \
    --non-strict \
    --preserve-environment PATH \
    --preserve-environment DISPLAY \
    --preserve-environment TMPDIR \
    --outdir /home/ks476/data_keith/encode4_duke//processing/starr_seq/240311_KS274_ASTARR_Output_Nextseq-pe-umis  \
    --no-container \
    /data/reddylab/software/cwl/GGR-cwl/v1.0/STARR-seq_pipeline/pipeline-pe-umis.cwl \
    /home/ks476/data_keith/encode4_duke//processing/starr_seq/jsons/starr_seq_download_metadata.240311_KS274_ASTARR_Output_Nextseq-pe-umis-${SLURM_ARRAY_TASK_ID}.json

# Delete any tmpdir not removed by cwltool
rm -rf /home/ks476/data_keith/encode4_duke//processing/starr_seq/tmpdirs/tmp-240311_KS274_ASTARR_Output_Nextseq-pe-umis-${SLURM_ARRAY_TASK_ID}-


Overwriting /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/240311_KS274_ASTARR_Output_Nextseq-pe-umis.sh


Execute SLURM array master file

In [10]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -p all \
 --depend afterok:$1 \
 --array 0-4%20 \
 /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/240311_KS274_ASTARR_Output_Nextseq-pe-umis.sh

Extract blocking job id

In [11]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create QC generating script

In [12]:
%%writefile /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/generate_qc_cell_240311_KS274_ASTARR_Output_Nextseq-pe-umis.sh
#!/bin/bash
#SBATCH --job-name=qc
#SBATCH --output=/home/ks476/data_keith/encode4_duke//processing/starr_seq/logs/qc_gen.240311_KS274_ASTARR_Output_Nextseq-pe-umis.out

source /data/reddylab/software/miniconda2/bin/activate alex
cd /home/ks476/data_keith/encode4_duke//processing/starr_seq/240311_KS274_ASTARR_Output_Nextseq-pe-umis

python /data/reddylab/software/cwl/bin/generate_stats_starrseq_paired_end.py ./ \
    -samples $(/bin/ls -1 *PBC.txt | sed 's@.PBC.txt@@') \
> qc.txt

Overwriting /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/generate_qc_cell_240311_KS274_ASTARR_Output_Nextseq-pe-umis.sh


Generate QCs for 240311_KS274_ASTARR_Output_Nextseq-pe-umis

In [13]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -p all \
 --depend afterok:$1 \
 /home/ks476/data_keith/encode4_duke//processing/starr_seq/scripts/generate_qc_cell_240311_KS274_ASTARR_Output_Nextseq-pe-umis.sh

Extract blocking job id

In [14]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)